<a href="https://colab.research.google.com/github/freguti/progettoML/blob/main/ProgettoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

STEP:

1 - Dovremo prendere, per ogni super categoria, tutte le annotazioni e associarle alla super categoria corrispondente.

2 -  fare il fine-tuning del modello di CTRL tramite le annotazioni associate alla super categoria e generare del testo

    2.1 - modficare i control code durante la fase di fine tuning, in modo tale da inserire i nostri.  

3 - Valutare il testo generato tramite una versione modificata di TexyGen.

Dubbi: 
- bisogna usare le categorie o le super categorie? 

Commenti:

Al momento ho creato una demo che mostra le super categorie e prende le annotazioni di una categoria fissa.

CTRL sono riuscito solo a farlo girare in modalità low memory, ma è molto più inefficiente

#Prepare environment

In [2]:
from google.colab import drive
from pycocotools.coco import COCO
import os
import subprocess

In [3]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoDemo.ipynb demo coco API
                   
dataDir='/content/drive/MyDrive/Machine Learning/Progetto'
dataType='val2017' 
modelDir = dataDir + '/model/' 
instFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)
#annFile = '{}/annotations/person_keypoints_{}.json'.format(dataDir,dataType)
annFile = '{}/annotations/captions_{}.json'.format(dataDir,dataType)

# Prepare Dataset

In [ ]:
# initialize COCO api for instance annotations
coco=COCO(instFile) #modello con le foto e le categorie
coco_ann=COCO(annFile) #modello con le annotazioni

loading annotations into memory...
Done (t=1.54s)
creating index...
index created!
loading annotations into memory...
Done (t=0.64s)
creating index...
index created!


In [ ]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

supernms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(supernms)))

COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush

COCO supercategories: 
electronic accessory appliance outdoor sports food person kitchen vehicle animal furniture indoor


In [ ]:
#(Carlo) Sto cercando di capire alcune cose, cella provvisoria (incompleto)
category_Id = {} # Dizionario - Chiave: categoria, Valore: Id della categoria
for elem in nms:
  category_Id[elem] = (coco.getCatIds(catNms=elem))
# print(category_Id)

category_ImagesIds = {} #Dizionario - Chiave: categoria, Valore: lista contentente gli id delle immagini di tale categoria
for (key,val) in category_Id.items():
  category_ImagesIds[key] = coco.getImgIds(catIds=val)

category_Annotations = {} #Dizionario - Chiave: categoria, Valore: lista di tutte le annotazioni di tutte le immagini di tale categoria
for (key, val) in category_ImagesIds.items():
  category_Annotations[key] = ids
  category_Annotations[key] = coco_ann.loadAnns(ids)

for (key, val) in category_Annotations.items():
  print(val)

In [ ]:
# load and display caption annotations
catIds = coco.getCatIds(catNms=['outdoor']); #seleziono le categorie che voglio analizzare
imgIds = coco.getImgIds(catIds=catIds ); #ottengo l'id delle immagini di categoria desiderata
annIds = coco_ann.getAnnIds(imgIds=imgIds); #ottengo l'annotazione relativa all'immagine con le categorie da me scelte [idAnn1,idAnn2,...] 
#TEST
#uso un id fisso
imgAnnIds = coco_ann.getAnnIds(imgIds=[324158,5698]); #ottengo l'annotazione relativa all'immagine con le categorie da me scelte
#FINE TEST
anns = coco_ann.loadAnns(annIds)
coco_ann.showAnns(anns)
#{'outdoor':[[caption1], [caption2]],'indoor':[[c1],[c2]]}

Streaming output truncated to the last 5000 lines.
A served tray filled with smaller plates of food
A lunch tray with multiple compartments filled with food.
Four plastic containers filled with food on a table.
A four compartment tray holding various food items.
A tray with carrots, snap beans, mash potatoes and an egg.
A tray with carrots, peas, rice, and a hard boiled egg.
Various foods served in a tray with different compartments.
A serving of rice, carrots, green pod beans and an egg each in their own small plastic containers.
Four containers of cooked rice, carrots, beans and an egg
Two skiers prepare to make their way past an embankment
A few people on skis skiing on a snowy mountain.
A man riding skis down a snow covered slope.
Two people on skis going down a small hill.
Two people cross country skiing in a valley.
A group of people standing on ski's in the snow.
some people standing next to each other on top of a snowy hill 
A group of men riding skis on top of a snow covered s

# CTRL

##Prepare model

In [21]:
def Patch_Keras():
  %cd '/content/drive/MyDrive/Machine Learning/Progetto/ctrl/'
  !patch -b /usr/local/lib/python2.7/dist-packages/tensorflow_estimator/python/estimator/keras.py estimator.patch
  %cd '/content/drive/MyDrive/Machine Learning/Progetto/ctrl/training_utils'

def init_no_API():
  %cd '/content/drive/MyDrive/Machine Learning/Progetto'
  !nvidia-smi
  !pip2 install tensorflow-gpu==1.14 #installo le dipendenze
  !rm -rf ctrl
  !git clone https://github.com/salesforce/ctrl #clono CTRL
  %cd '/content/drive/MyDrive/Machine Learning/Progetto/ctrl'
  if lowerMemory == True:
    !patch -b /usr/local/lib/python2.7/dist-packages/tensorflow_estimator/python/estimator/keras.py estimator.patch
    !git checkout lower_memory
  else:
    !pip install torch
    !git checkout master
  !pip2 install fastBPE

  
  if os.path.isfile("/content/drive/MyDrive/Machine Learning/Progetto/seqlen256_v1.ckpt/model.ckpt-413000.data-00000-of-00001") == False:
    %cd '/content/drive/MyDrive/Machine Learning/Progetto'
    !pip2 install gsutil
    !gsutil -m cp -r gs://sf-ctrl/seqlen256_v1.ckpt . #48 layer
    #!gsutil -m cp -r gs://sf-ctrl/seqlen256_36layers_v0.ckpt/ . #36 layer

def generate_no_API():
  #from keras import backend as K
  #config = tf.ConfigProto()
  #config.gpu_options.allow_growth = True
  #config.log_device_placement = True
  #session = tf.Session(config=config)
  #K.set_session(session)
  %cd '/content/drive/MyDrive/Machine Learning/Progetto/ctrl'
  if lowerMemory == True:
    !python2 generation.py --model ../seqlen256_v1.ckpt/model.ckpt-413000.data-00000-of-00001
  else:
    !python2 pytorch_generation.py --temperature 0.5 --model ../seqlen256_v1.ckpt/model.ckpt-413000.data-00000-of-00001
  #K.get_session().close();

def init_API():
  !pip install transformers
  from transformers import CTRLModel, CTRLConfig, CTRLTokenizer
  import torch
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  #print("1")
  #configuration = CTRLConfig(n_layer = 32)
  #print("2")
  #model = CTRLModel(configuration)
  print("1")
  tokenizer = CTRLTokenizer.from_pretrained('ctrl')
  print("2")
  model = CTRLModel.from_pretrained('ctrl', config = CTRLConfig(n_layer = 32, diff = 7000))
  print("3")
  model.to(device)
  print("4")
  return model,tokenizer

def generate_API(model,tokenizer):
  inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
  print("5")
  #outputs = model(**inputs)
  print("6")
  last_hidden_states = outputs.last_hidden_state
  print("7")
  return outputs

def finetune_CTRL_API():
  #farò il finetune e salverò il modello su drive, cancellando la cartella del API = false
  #nel metodo API = False dovrò cancellare questo modello
  i = 50

def Create_tf_records(file_list = [("moby","moby_dick.txt")]):
  #avrò una lista con [(nome_label,path_file),...]. dovrò leggere path_file da google drive e produrrò una lista con il path al file contenente il tf record
  Patch_Keras()
  os.system("wget -O moby_dick.txt https://www.gutenberg.org/files/2701/2701-0.txt") #DEBUG, per le prove finchè non avrò la lista
  #mi salva i record in moby_dick.txt.tfrecords
  for (control_f,path_f) in file_list:
    path_file = "/content/drive/MyDrive/Machine Learning/Progetto/ctrl/training_utils/" + path_f + ".tfrecords"
    if os.path.isfile(path_file) == False:
      #se non esiste il file tfrecords devo crearlo
      command_create_tf = "python2 make_tf_records.py --text_file {0} --control_code {1} --sequence_len 256".format(path_f,control_f)
      output = subprocess.check_output(command_create_tf,shell=True)
  #creo una lista con (control_code,tf_record)
  record_list = [(i,j.replace('.txt','.txt.tfrecords')) for i,j in file_list]
  print(record_list)
  return record_list

def finetune_CTRL_no_API(file_list = []):
  record_list = Create_tf_records()
  
  

## Fine tuning CTRL

## Text generation

In [22]:
use_api = False
lowerMemory = False
if use_api == False:
  init_no_API()
  finetune_CTRL_no_API()
  #generate_no_API()
else:
  model,tokenizer = init_API()
  output = generate_API(model = model,tokenizer = tokenizer)
  print(output)


/content/drive/MyDrive/Machine Learning/Progetto
Thu May 27 09:30:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                       

In [8]:

!ls -la  /usr/local/lib/python2.7/dist-packages/tensorflow_estimator/python/estimator

total 632
drwxr-sr-x 9 root staff   4096 May 27 08:52 .
drwxr-sr-x 3 root staff   4096 May 27 08:52 ..
drwxr-sr-x 4 root staff   4096 May 27 08:52 api
drwxr-sr-x 5 root staff   4096 May 27 08:52 canned
-rw-r--r-- 1 root staff  19986 May 27 08:52 early_stopping.py
-rw-r--r-- 1 root staff  21231 May 27 08:52 early_stopping.pyc
-rw-r--r-- 1 root staff   4238 May 27 08:52 estimator_lib.py
-rw-r--r-- 1 root staff   3484 May 27 08:52 estimator_lib.pyc
-rw-r--r-- 1 root staff 100728 May 27 08:52 estimator.py
-rw-r--r-- 1 root staff  89295 May 27 08:52 estimator.pyc
drwxr-sr-x 2 root staff   4096 May 27 08:52 export
-rw-r--r-- 1 root staff  19191 May 27 08:52 exporter.py
-rw-r--r-- 1 root staff  20189 May 27 08:52 exporter.pyc
-rw-r--r-- 1 root staff   4888 May 27 08:52 extenders.py
-rw-r--r-- 1 root staff   4791 May 27 08:52 extenders.pyc
-rw-r--r-- 1 root staff   6256 May 27 08:52 gc.py
-rw-r--r-- 1 root staff   7451 May 27 08:52 gc.pyc
drwxr-sr-x 2 root staff   4096 May 27 08:52 head
drwxr-

# Texygen evaluation

In [ ]:
%cd '/content/drive/MyDrive/Machine Learning/Progetto'
!git clone https://github.com/geek-ai/Texygen.git
!pip install colorama
%cd '/content/drive/MyDrive/Machine Learning/Progetto/Texygen'
# run SeqGAN with default setting
!python3 main.py